In [1]:
import nltk
import re

In [2]:
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\ARWA\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
from nltk.corpus import twitter_samples

In [4]:
import pandas as pd
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')


In [5]:
print(positive_tweets)

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)', '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!', '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!', '@97sides CONGRATS :)', 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days', '@BhaktisBanter @PallaviRuhail This one is irresistible :)\n#FlipkartFashionFriday http://t.co/EbZ0L2VENM', "We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI", '@Impatientraider On second thought, there’s just not enough time for a DD :) But new shorts entering system. Sheep must be buying.', 'Jgh , but we have to go to Bayan :D bye', 'As an act of mischievousness, am calling the ETL layer of our in-house warehousing app Katam

In [6]:
print(positive_tweets[0])
original_tweet=positive_tweets[0]

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [7]:
#remove #hastags
tweet=re.sub(r'#+\S+',"",original_tweet)
tweet

' @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'

In [8]:
#remove @
tweet=re.sub(r'@+\S+',"",tweet)
tweet

'    for being top engaged members in my community this week :)'

In [9]:
#remove links
print(positive_tweets[5])
tweet=re.sub(r'https?://\S+',"",positive_tweets[5])
print(tweet)
#remove any character which is not number or alphabet
tweet=re.sub(r'[^\w\s]',"",tweet)
tweet

@BhaktisBanter @PallaviRuhail This one is irresistible :)
#FlipkartFashionFriday http://t.co/EbZ0L2VENM
@BhaktisBanter @PallaviRuhail This one is irresistible :)
#FlipkartFashionFriday 


'BhaktisBanter PallaviRuhail This one is irresistible \nFlipkartFashionFriday '

In [10]:
#remove \n in text
tweet=re.sub(r'\n',"",tweet)
tweet

'BhaktisBanter PallaviRuhail This one is irresistible FlipkartFashionFriday '

In [11]:
print(positive_tweets[8])
#remove emojis
tweet=re.sub(r':\S*',"",positive_tweets[8])
tweet

Jgh , but we have to go to Bayan :D bye


'Jgh , but we have to go to Bayan  bye'

In [12]:
def clean_text(tweet):
    tweet=re.sub(r'#+\S+',"",tweet)  #remove hashtags
    #remove @
    tweet=re.sub(r'@+\S+',"",tweet)
    #remove links
    tweet=re.sub(r'https?://\S+',"",tweet)
    #remove \n in text
    tweet=re.sub(r'\n',"",tweet)
    #remove emojis
    tweet=re.sub(r':\S*',"",tweet)
    #remove phone numbers
    tweet=re.sub(r'\s\d+\s',"",tweet)
    #remove any character which is not number or alphabet
    tweet=re.sub(r'[^\w\s]',"",tweet)
    #remove space at the beginning
    tweet=re.sub("^\s+","",tweet)
    #remove space at the end
    tweet=re.sub("\s+$","",tweet)
    return tweet
    

In [13]:
clean_positive_tweets=[clean_text(x) for x in positive_tweets ]

In [14]:
clean_positive_tweets

['for being top engaged members in my community this week',
 'Hey James How odd  Please call our Contact Centre onand we will be able to assist you  Many thanks',
 'we had a listen last night  As You Bleed is an amazing track When are you in Scotland',
 'CONGRATS',
 'yeaaaah yippppy  my accnt verified rqst has succeed got a blue tick mark on my fb profile  indays',
 'This one is irresistible',
 'We dont like to keep our lovely customers waiting for long We hope you enjoy Happy Friday  LWWF',
 'On second thought theres just not enough time for a DD  But new shorts entering system Sheep must be buying',
 'Jgh  but we have to go to Bayan  bye',
 'As an act of mischievousness am calling the ETL layer of our inhouse warehousing app KatamariWell as the name implies',
 'for being top influencers in my community this week',
 'Who Wouldnt Love These BigJuicySelfies',
 'follow  amp  follow u back',
 'Perfect so you already know whats waiting for you',
 'Great new opportunity for junior triathlet

In [15]:
#convert tweets to lower text 
clean_positive_tweets=[x.lower() for x in clean_positive_tweets]

In [16]:
clean_positive_tweets

['for being top engaged members in my community this week',
 'hey james how odd  please call our contact centre onand we will be able to assist you  many thanks',
 'we had a listen last night  as you bleed is an amazing track when are you in scotland',
 'congrats',
 'yeaaaah yippppy  my accnt verified rqst has succeed got a blue tick mark on my fb profile  indays',
 'this one is irresistible',
 'we dont like to keep our lovely customers waiting for long we hope you enjoy happy friday  lwwf',
 'on second thought theres just not enough time for a dd  but new shorts entering system sheep must be buying',
 'jgh  but we have to go to bayan  bye',
 'as an act of mischievousness am calling the etl layer of our inhouse warehousing app katamariwell as the name implies',
 'for being top influencers in my community this week',
 'who wouldnt love these bigjuicyselfies',
 'follow  amp  follow u back',
 'perfect so you already know whats waiting for you',
 'great new opportunity for junior triathlet

In [17]:
#Do the same for negative tweets
clean_negative_tweets=[clean_text(x) for x in negative_tweets]
clean_negative_tweets=[x.lower() for x in clean_negative_tweets]
clean_negative_tweets

['hopeless for tmr',
 'everything in the kids section of ikea is so cute shame im nearlyinmonths',
 'that heart sliding into the waste basket',
 'i hate japanese call him bani   too',
 'dang starting next week i have work',
 'oh god my babies faces',
 'make me smile',
 'work neighbour on motors asked why and he said hates the updates on search',
 'why sialan',
 'athabasca glacier was there in',
 'i have a really good mampg idea but im never going to meet them',
 'mare ivan',
 'happy trip keep safe see you soon',
 'im so tired hahahah',
 'with knee replacements they get you up amp about the same day    ouch',
 'relate to the sweet n sour kind of bipolar people in your life cuz my life is full of them',
 'pleasse',
 'im not sure tho',
 'i feel stupidi just cant seem to grasp the basics of digital painting and nothing ive been researching is helping any',
 'good lord',
 'i feel lonely someone talk to me guys and girls',
 'no assignment but we have project  really',
 'just want to play vid

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ARWA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

In [21]:
#process the +tweets by doing the following
#1- remove stop words
#2- pass the words through the stemmer 
def process_text(tweets):
    clean_tweets=[]
    for tweet in tweets:
        no_stop_words= [x for x in tweet.split() if x not in stop_words]
        clean_tweet=[stemmer.stem(x) for x in no_stop_words]
        clean_tweets.append(clean_tweet)
    return clean_tweets



In [22]:
#clean positive tweets
clean_positive_tweets=process_text(clean_positive_tweets)
clean_positive_tweets

[['top', 'engag', 'member', 'commun', 'week'],
 ['hey',
  'jame',
  'odd',
  'pleas',
  'call',
  'contact',
  'centr',
  'onand',
  'abl',
  'assist',
  'mani',
  'thank'],
 ['listen', 'last', 'night', 'bleed', 'amaz', 'track', 'scotland'],
 ['congrat'],
 ['yeaaaah',
  'yippppi',
  'accnt',
  'verifi',
  'rqst',
  'succeed',
  'got',
  'blue',
  'tick',
  'mark',
  'fb',
  'profil',
  'inday'],
 ['one', 'irresist'],
 ['dont',
  'like',
  'keep',
  'love',
  'custom',
  'wait',
  'long',
  'hope',
  'enjoy',
  'happi',
  'friday',
  'lwwf'],
 ['second',
  'thought',
  'there',
  'enough',
  'time',
  'dd',
  'new',
  'short',
  'enter',
  'system',
  'sheep',
  'must',
  'buy'],
 ['jgh', 'go', 'bayan', 'bye'],
 ['act',
  'mischiev',
  'call',
  'etl',
  'layer',
  'inhous',
  'wareh',
  'app',
  'katamariwel',
  'name',
  'impli'],
 ['top', 'influenc', 'commun', 'week'],
 ['wouldnt', 'love', 'bigjuicyselfi'],
 ['follow', 'amp', 'follow', 'u', 'back'],
 ['perfect', 'alreadi', 'know', 'w

In [23]:
#clean negative tweets
clean_negative_tweets=process_text(clean_negative_tweets)
clean_negative_tweets

[['hopeless', 'tmr'],
 ['everyth', 'kid', 'section', 'ikea', 'cute', 'shame', 'im', 'nearlyinmonth'],
 ['heart', 'slide', 'wast', 'basket'],
 ['hate', 'japanes', 'call', 'bani'],
 ['dang', 'start', 'next', 'week', 'work'],
 ['oh', 'god', 'babi', 'face'],
 ['make', 'smile'],
 ['work', 'neighbour', 'motor', 'ask', 'said', 'hate', 'updat', 'search'],
 ['sialan'],
 ['athabasca', 'glacier'],
 ['realli', 'good', 'mampg', 'idea', 'im', 'never', 'go', 'meet'],
 ['mare', 'ivan'],
 ['happi', 'trip', 'keep', 'safe', 'see', 'soon'],
 ['im', 'tire', 'hahahah'],
 ['knee', 'replac', 'get', 'amp', 'day', 'ouch'],
 ['relat',
  'sweet',
  'n',
  'sour',
  'kind',
  'bipolar',
  'peopl',
  'life',
  'cuz',
  'life',
  'full'],
 ['pleass'],
 ['im', 'sure', 'tho'],
 ['feel',
  'stupidi',
  'cant',
  'seem',
  'grasp',
  'basic',
  'digit',
  'paint',
  'noth',
  'ive',
  'research',
  'help'],
 ['good', 'lord'],
 ['feel', 'lone', 'someon', 'talk', 'guy', 'girl'],
 ['assign', 'project', 'realli'],
 ['want',

In [24]:
total_negative_tweets=len(clean_negative_tweets)
total_negative_tweets

5000

In [25]:
total_positive_tweets=len(clean_positive_tweets)
total_positive_tweets

5000

In [26]:
all_tweets=[]
all_tweets.extend(clean_positive_tweets)
all_tweets.extend(clean_negative_tweets)

In [27]:
#checking if positive and negative tweets got appended by chekcing the length
len(all_tweets)


10000

In [28]:
#creating array of positive and negative labels
positive_labels=[1]*len(clean_positive_tweets)
negative_labels=[0]*len(clean_negative_tweets)

In [29]:
labels=[]
labels.extend(positive_labels)
labels.extend(negative_labels)

In [30]:
#shuffle the data
labels
import random
zip_list = list((zip(all_tweets, labels)))
random.shuffle(zip_list)
tweets, labels = zip(*zip_list)

In [31]:
def build_freq_dict(tweets,labels):
    freq_dict=dict()
    #{"word":[count of positive,count of negative]}
    for i,tweet in enumerate(tweets):
        for word in tweet:
            if word not in freq_dict.keys():
                if labels[i]==0:
                    freq_dict[word]=[0,1]
                else:
                    freq_dict[word]=[1,0]
            else:
                if labels[i]==0:
                    freq_dict[word][1]+=1
                else:
                    freq_dict[word][0]+=1
    return freq_dict





In [32]:
freq_dict=build_freq_dict(tweets,labels)

In [33]:
freq_dict

{'love': [396, 151],
 'nicknam': [1, 1],
 'babygirl': [1, 0],
 'femal': [2, 3],
 'wanna': [28, 94],
 'get': [206, 231],
 'poppin': [0, 1],
 'joc': [0, 1],
 'playin': [0, 1],
 'agh': [0, 2],
 'sorri': [19, 148],
 'mayb': [18, 27],
 'ill': [98, 57],
 'surpris': [9, 5],
 'brill': [1, 1],
 'achiev': [2, 0],
 'togeth': [6, 7],
 'thank': [636, 106],
 'rashmi': [1, 0],
 'time': [122, 161],
 'dedic': [2, 1],
 'complet': [10, 9],
 'inspir': [6, 0],
 'sound': [23, 18],
 'fun': [57, 32],
 'miss': [25, 298],
 'lil': [4, 8],
 'shawti': [0, 1],
 'work': [110, 130],
 'today': [111, 109],
 'better': [52, 36],
 'tomorrow': [40, 35],
 'blue': [9, 5],
 'eye': [13, 12],
 'look': [137, 96],
 'caught': [4, 1],
 'trust': [9, 2],
 'well': [99, 62],
 'awel': [0, 1],
 'long': [35, 38],
 'drunk': [1, 2],
 'care': [17, 19],
 'ok': [35, 33],
 'good': [236, 99],
 'hear': [33, 33],
 'keep': [66, 34],
 'even': [35, 69],
 'dont': [95, 232],
 'want': [96, 244],
 'anyth': [22, 20],
 'happen': [16, 51],
 'bae': [4, 12],


In [34]:
def get_total_pos_neg_frequency(freq_dict):
    pos_frequency=0
    neg_frequency=0
    for key,value in freq_dict.items():
        pos_frequency+=value[0]
        neg_frequency+=value[1]
    return pos_frequency,neg_frequency


In [35]:
pos_freq,neg_freq=get_total_pos_neg_frequency(freq_dict)

In [36]:
#build probability
#P(word i |pos) and P(word i|neg)
#we will use this formula P(word i|class)=freq(word,class)+1 / Number of words in class + number of unique words
def build_probability(freq_dict,count_pos,count_neg):
    total_unique_words=len(freq_dict.keys())
    probability_dict={}
    for word, freq in freq_dict.items():
        probability_dict[word]=[((freq[1]+1)/(count_pos+total_unique_words)),((freq[0]+1)/(count_neg+total_unique_words))]
    return probability_dict


In [37]:
probability_dict=build_probability(freq_dict,pos_freq,neg_freq)

In [38]:
probability_dict

{'love': [0.004159597175852444, 0.010979589579069639],
 'nicknam': [5.473154178753216e-05, 5.531279384921732e-05],
 'babygirl': [2.736577089376608e-05, 5.531279384921732e-05],
 'femal': [0.00010946308357506431, 8.296919077382599e-05],
 'wanna': [0.0025997482349077775, 0.0008020355108136512],
 'get': [0.00634885884735373, 0.005724874163393993],
 'poppin': [5.473154178753216e-05, 2.765639692460866e-05],
 'joc': [5.473154178753216e-05, 2.765639692460866e-05],
 'playin': [5.473154178753216e-05, 2.765639692460866e-05],
 'agh': [8.209731268129824e-05, 2.765639692460866e-05],
 'sorri': [0.004077499863171146, 0.0005531279384921732],
 'mayb': [0.0007662415850254501, 0.0005254715415675646],
 'ill': [0.0015872147118384326, 0.0027379832955362576],
 'surpris': [0.00016419462536259647, 0.0002765639692460866],
 'brill': [5.473154178753216e-05, 5.531279384921732e-05],
 'achiev': [2.736577089376608e-05, 8.296919077382599e-05],
 'togeth': [0.00021892616715012863, 0.00019359477847226064],
 'thank': [0.00

In [39]:
import numpy as np

In [42]:
#calculate naive bayes inference
#summation of log(P(w|pos)/P(w|neg) )
#also calculate log prior = log(P(pos)/p(neg))
def build_naive_inference(probability_dict,tweets,count_pos,count_negative):
    predictions=[]
    for tweet in tweets:
        result=np.log(count_pos/count_negative)
        for word in tweet:
            result+=np.log(probability_dict[word][1]/probability_dict[word][0])
        if result>=0:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions




In [43]:
predictions=build_naive_inference(probability_dict,tweets,pos_freq,neg_freq)

In [44]:
predictions

[1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,


In [45]:
labels

(1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,


In [46]:
from sklearn.metrics import accuracy_score

In [47]:
acc=accuracy_score(labels,predictions)

In [48]:
#Calculate model accuracy
acc

0.8727

In [49]:
#To test with new test cases we need a predict function that cleans text first and uses the inference function to get lablel
def naive_predict(test_tweet):
    #convert to lowercase
    test_tweet=test_tweet.lower()
    test_tweet=clean_text(test_tweet)
    processed_tweet=process_text([test_tweet])
    predictions=build_naive_inference(probability_dict,processed_tweet,pos_freq,neg_freq)
    return predictions[0]
    

In [50]:
naive_predict("I am very happy today")

1

In [53]:
naive_predict("My day was bad")

0